0) Извлечение данных 

In [ ]:
import pandas as pd

bike_data_df = pd.read_csv("BikeData.csv", sep=',')
bike_data_df.columns = bike_data_df.columns.str.strip()

bike_data_df


1) Создайте новую переменную Total_count, в которую записать общее количество
арендованных велосипедов за каждый час.

In [ ]:
bike_data_df["Total count"] = bike_data_df["Partner 1"] + bike_data_df["Partner 2"] 

bike_data_df

2) Удалите ряды Partner 1 и Partner 2

In [ ]:
bike_data_df = bike_data_df.drop(columns=["Partner 1", "Partner 2"])

bike_data_df


3) Преобразуйте переменную Date к стандартной форме (to_datetime).

In [ ]:
bike_data_df["Date"] = pd.to_datetime(bike_data_df["Date"], errors="coerce", format="%d/%m/%Y")

bike_data_df

4) Определите сколько велосипедов было выдано в день, месяц и число которого
совпадает с днем выполнения этого задания. Определите, какой это был день недели.

In [ ]:
from datetime import datetime

current_date = pd.to_datetime(datetime.today(), format="%d/%m/%Y")

date_mask = (bike_data_df["Date"].dt.day == current_date.day) & (bike_data_df["Date"].dt.month == current_date.month)
today_trimmed_df = bike_data_df[date_mask]

today_trimmed_df

total_bikes_by_current_date_df = pd.DataFrame([{
    "Date": current_date.date(),
    "Day_of_week": today_trimmed_df["Date"].dt.day_name().iloc[0],
    "Total_Bikes": today_trimmed_df["Total count"].sum()
}])

total_bikes_by_current_date_df
 


5) Перекодируйте категориальную переменную Functioning Day(рабочий
или нерабочий день) в переменную типа boolean, т.е. "Yes"=True и "No" = False. Выведите
количество нерабочих дней ( не часов ). День считается нерабочим, если пункт выдачи
велосипедов работал менее 12 часов.

In [ ]:

bike_data_df["Functioning Day"] = bike_data_df["Functioning Day"] == "Yes"

all_pseudo_working_days = bike_data_df.groupby(bike_data_df["Date"].dt.date)["Functioning Day"].sum()

actual_working_days_count = all_pseudo_working_days[all_pseudo_working_days < 12].count()
actual_working_days_count




6) Перекодируйте переменную Holiday (являлся ли день праздничным) в 0, если
No Holiday, и 1, если Holiday. (используйте анонимную функцию lambda).

In [ ]:
bike_data_df["Holiday"] = bike_data_df["Holiday"].apply(lambda val: True if val == "No Holiday" else False )

bike_data_df

7) Введите новую категориальную переменную Temperature category, которая будет
равна:

"Freezing", если температура < 0;
"Chilly", если 0 <= температура < 15;
"Nice", если 15 <= температура < 26;
"Hot", если 26 <= температура.

In [ ]:
def encode_temperature(t):
    if t < 0:
        return "Freezing"
    elif t < 15:
        return "Chilly"
    elif t < 26:
        return "Nice"
    else:
        return "Hot"
    
bike_data_df["Temperature category"] = bike_data_df["Temperature"].apply(lambda t: encode_temperature(t))

bike_data_df

8) Создайте новую переменную Good weather, которая будет равна 1 (т.е. хорошая
погода), если:
Temperature category = "Nice",
Humidity в диапазоне от 40 до 60 ,
скорость ветра Wind speed меньше 5.4,
нет дождя (Rainfall)
нет снега (Snowfall).
Определите, сколько процентов наблюдений соответствует хорошей погоде.

In [ ]:
cond_mask = (
    (bike_data_df["Temperature category"] == "Nice") &
    (bike_data_df["Humidity"].between(40, 60)) &
    (bike_data_df["Wind speed"] < 5.4) &
    (bike_data_df["Rainfall"] == 0) &
    (bike_data_df["Snowfall"] == 0)
)

bike_data_df["Good_conditions"] = cond_mask

bike_data_df["Good_conditions"].sum() / len(bike_data_df) * 100

9) Сгруппируйте данные по сезонам (Seasons), внутри сезона по категории погоды
(Temperature category) и вывести для полученной группировки количество арендованных
велосипедов. Вывести результаты в виде сгруппированной таблицы (через groupby) и в
виде сводной таблицы (pivot_table).

In [ ]:
seasons_temperature_group = bike_data_df.groupby(["Seasons", "Temperature category"])["Total count"].sum()

pivot_result = bike_data_df.pivot_table(
    index="Seasons",
    columns="Temperature category",
    values="Total count",
    aggfunc="sum"
)

display(seasons_temperature_group)
display(pivot_result)

10) Определите сезон и температуру, при которых выдается наибольшее и наименьшее
количество велосипедов.

In [ ]:
grouped = bike_data_df.groupby(
    ["Seasons", "Temperature category"]
)["Total count"].sum()

max_value = grouped.max()
min_value = grouped.min()

max_combo = grouped[max_value]
min_combo = grouped[min_value]

print("Максимум:")
print(max_combo)

print("\nМинимум:")
print(min_combo)
